In [3]:
#Trains neural network using GSE63990_series_matrix.txt and homebrew code

In [1]:
#imports all needed modules 
import NotebookFinder
import data_cleaner as dclean
import numpy as np
import pandas as pd
import MLP_Clean as NN
import gene_selection as gs

#import feature selection modules
import sklearn.feature_selection 


#lets try mnist

from sklearn.datasets import fetch_openml
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

importing Jupyter notebook from data_cleaner.ipynb
importing data_cleaner modules...
done
importing Jupyter notebook from MLP_Clean.ipynb
importing Jupyter notebook from gene_selection.ipynb


In [2]:
#data organization/reformatting of GSE63990_series_matrix.txt 

#returns characteristics = X
        #expression = Y
        # 'non-infectious illness': 0, 'bacterial': 1, 'viral': 2

#call on data_cleaner to open and formate GSE63990_series_matrix
data = dclean.clean_GSE_series_matrix('GSE63990_series_matrix.txt')
# X_train, Y_train, X_test, Y_test = gs.select(data, 10)



reading file...
Dropping Series_title
Dropping Series_geo_accession
Dropping Series_status
Dropping Series_submission_date
Dropping Series_last_update_date
Dropping Series_pubmed_id
Dropping Series_summary
Dropping Series_overall_design
Dropping Series_type
Dropping Series_contributor
Dropping Series_sample_id
Dropping Series_contact_name
Dropping Series_contact_email
Dropping Series_contact_department
Dropping Series_contact_institute
Dropping Series_contact_address
Dropping Series_contact_city
Dropping Series_contact_state
Dropping Series_contact_zip/postal_code
Dropping Series_contact_country
Dropping Series_supplementary_file
Dropping Series_platform_id
Dropping Series_platform_taxid
Dropping Series_sample_taxid
Dropping Series_relation
Converting Sample_characteristics_ch1


In [3]:
def round_param(parameters):
    #rounding to nearest integer doesn't work
    #round to 1 decimal place does work
    W1=np.round(parameters['W1'],1)
    W2=np.round(parameters['W2'],1)
    b1=np.round(parameters['b1'],1)
    b2=np.round(parameters['b2'],1)   
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return(parameters)

In [4]:
#mnist test to make sure my model isn't trash 
#spoiler it's not
def MNIST_get(train_size, test_size):
    train_samples = train_size

    X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

    random_state = check_random_state(0)
    permutation = random_state.permutation(X.shape[0])
    X = X[permutation]
    y = y[permutation]
    X = X.reshape((X.shape[0], -1))

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_samples, test_size=test_size)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    #transpose since shape is wrong
    X_train, X_test, Y_train, Y_test = X_train.T, X_test.T, y_train.T, y_test.T

    #onehotencode the Y for mnist
    ohe = OneHotEncoder()

    Y_train = Y_train.reshape(-1, 1)
    Y_train = ohe.fit_transform(Y_train).toarray()

    Y_test= Y_test.reshape(-1, 1)
    Y_test = ohe.fit_transform(Y_test).toarray()
    
    return X_train, Y_train, X_test, Y_test

In [24]:
#find the best models with rounding
hidden_layer_sizes = np.arange(1,10)
gene_list = np.arange(1,20)
for gene_number in gene_list:
    X_train, Y_train, X_test, Y_test = gs.select(data, gene_number)
    #lets log2 this
    X_train = np.log2(X_train)
    X_test = np.log2(X_test)
    for n_h in hidden_layer_sizes:
        parameters = NN.nn_model(X_train, Y_train, n_h, num_iterations =10000, seed=42)
        #round the weights after training
        parameters = round_param(parameters)
        test_predictions = NN.predict(parameters, X_test)
        test_labels = np.argmax(Y_test, axis = 0)
        test_t = (test_labels-test_predictions)
        test_accuracy = np.sum(np.where(test_t == 0, 1, 0))/Y_test.shape[1]*100
        train_predictions = NN.predict(parameters, X_train)
        train_labels = np.argmax(Y_train, axis = 0)
        train_t = (train_labels-train_predictions)
        train_accuracy = np.sum(np.where(train_t == 0, 1, 0))/Y_train.shape[1]*100
        if test_accuracy > 70 or train_accuracy > 70:
            print ("with {} gene(s)".format(gene_number))
            print ("Accuracy for {} hidden units: {} / {} %".format(n_h, train_accuracy, test_accuracy))

/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376


/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
6852                 207329_at"  51.908816
with 2 gene(s)
Accuracy for 4 hidden units: 65.87301587301587 / 75.0 %
with 2 gene(s)
Accuracy for 6 hidden units: 66.26984126984127 / 75.0 %
with 2 gene(s)
Accuracy for 8 hidden units: 68.65079365079364 / 71.42857142857143 %
with 2 gene(s)
Accuracy for 9 hidden units: 67.46031746031747 / 71.42857142857143 %


/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


...

                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
6852                 207329_at"  51.908816
21443              222084_s_at"  45.796933
12152              212768_s_at"  45.511495
10952              211548_s_at"  39.885743
3439               203913_s_at"  39.213689
19338              219975_x_at"  38.786768
9477                 209992_at"  36.792002
5370                 205844_at"  36.757702
5702               206177_s_at"  35.538623
19032                219669_at"  35.401498
3440               203914_x_at"  35.133532
8786               209293_x_at"  34.937390
22240  AFFX-HUMRGE/M10098_M_at"  34.897467
16040              216671_x_at"  31.861675
9445                 209960_at"  28.876673
16806                217442_at"  28.460255
with 17 gene(s)
Accuracy for 2 hidden units: 70.23809523809523 / 67.85714285714286 %
with 17 gene(s)
Accuracy for 3 hidden units: 69.04761904761905 / 75.0 %
with 17 gene(s)
Accuracy for 8 hidden units: 72.61904761904762 / 75.0

/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
6852                 207329_at"  51.908816
21443              222084_s_at"  45.796933
12152              212768_s_at"  45.511495
10952              211548_s_at"  39.885743
3439               203913_s_at"  39.213689
19338              219975_x_at"  38.786768
9477                 209992_at"  36.792002
5370                 205844_at"  36.757702
5702               206177_s_at"  35.538623
19032                219669_at"  35.401498
3440               203914_x_at"  35.133532
8786               209293_x_at"  34.937390
22240  AFFX-HUMRGE/M10098_M_at"  34.897467
16040              216671_x_at"  31.861675
9445                 209960_at"  28.876673
16806                217442_at"  28.460255
2680                 203153_at"  27.864647
with 18 gene(s)
Accuracy for 7 hidden units: 68.65079365079364 / 75.0 %


/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
6852                 207329_at"  51.908816
21443              222084_s_at"  45.796933
12152              212768_s_at"  45.511495
10952              211548_s_at"  39.885743
3439               203913_s_at"  39.213689
19338              219975_x_at"  38.786768
9477                 209992_at"  36.792002
5370                 205844_at"  36.757702
5702               206177_s_at"  35.538623
19032                219669_at"  35.401498
3440               203914_x_at"  35.133532
8786               209293_x_at"  34.937390
22240  AFFX-HUMRGE/M10098_M_at"  34.897467
16040              216671_x_at"  31.861675
9445                 209960_at"  28.876673
16806                217442_at"  28.460255
2680                 203153_at"  27.864647
5975                 206450_at"  27.230384
with 19 gene(s)
Accuracy for 8 hidden units: 70.23809523809523 / 78.57142857142857 %


In [40]:
# try select models with rounding
randomseed = 42
number_hidden = 8
gene_number = 19

# X_train, Y_train, X_test, Y_test = MNIST_get(5000,10000)
X_train, Y_train, X_test, Y_test = gs.select(data, gene_number)
#lets log2 this
X_train = np.log2(X_train)
X_test = np.log2(X_test)
parameters = NN.nn_model(X_train, Y_train, n_h = number_hidden, num_iterations = 6000, print_cost=True, seed=randomseed)
parameters = round_param(parameters)
predictions = NN.predict(parameters, X_train)
labels = np.argmax(Y_train, axis = 0)
t = (labels-predictions)      
accuracy = np.sum(np.where(t == 0, 1, 0))/Y_train.shape[1]*100
print('train accuracy:', accuracy,'%')
predictions = NN.predict(parameters, X_test)
labels = np.argmax(Y_test, axis = 0)
t = (labels-predictions)      
accuracy = np.sum(np.where(t == 0, 1, 0))/Y_test.shape[1]*100
print('test accuracy:', accuracy,'%')

/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
6852                 207329_at"  51.908816
21443              222084_s_at"  45.796933
12152              212768_s_at"  45.511495
10952              211548_s_at"  39.885743
3439               203913_s_at"  39.213689
19338              219975_x_at"  38.786768
9477                 209992_at"  36.792002
5370                 205844_at"  36.757702
5702               206177_s_at"  35.538623
19032                219669_at"  35.401498
3440               203914_x_at"  35.133532
8786               209293_x_at"  34.937390
22240  AFFX-HUMRGE/M10098_M_at"  34.897467
16040              216671_x_at"  31.861675
9445                 209960_at"  28.876673
16806                217442_at"  28.460255
2680                 203153_at"  27.864647
5975                 206450_at"  27.230384
Cost after iteration 0: 279.409897
Cost after iteration 1000: 204.469750
Cost after iteration 2000: 194.210618
Cost after iteration 3000: 19

In [42]:
#try with other random splits of the dataset
X_train, Y_train, X_test, Y_test = gs.select(data, gene_number, seed=100)
#lets log2 this
X_train = np.log2(X_train)
X_test = np.log2(X_test)
predictions = NN.predict(parameters, X_train)
labels = np.argmax(Y_train, axis = 0)
t = (labels-predictions)      
accuracy = np.sum(np.where(t == 0, 1, 0))/Y_train.shape[1]*100
print('train accuracy:', accuracy,'%')
predictions = NN.predict(parameters, X_test)
labels = np.argmax(Y_test, axis = 0)
t = (labels-predictions)      
accuracy = np.sum(np.where(t == 0, 1, 0))/Y_test.shape[1]*100
print('test accuracy:', accuracy,'%')

/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
6852                 207329_at"  51.908816
21443              222084_s_at"  45.796933
12152              212768_s_at"  45.511495
10952              211548_s_at"  39.885743
3439               203913_s_at"  39.213689
19338              219975_x_at"  38.786768
9477                 209992_at"  36.792002
5370                 205844_at"  36.757702
5702               206177_s_at"  35.538623
19032                219669_at"  35.401498
3440               203914_x_at"  35.133532
8786               209293_x_at"  34.937390
22240  AFFX-HUMRGE/M10098_M_at"  34.897467
16040              216671_x_at"  31.861675
9445                 209960_at"  28.876673
16806                217442_at"  28.460255
2680                 203153_at"  27.864647
5975                 206450_at"  27.230384
train accuracy: 72.61904761904762 %
test accuracy: 71.42857142857143 %


In [ ]:
# Gene      Score
# 22239  AFFX-HUMRGE/M10098_5_at"  62.413376
# 6852                 207329_at"  51.908816
# 21443              222084_s_at"  45.796933
# 12152              212768_s_at"  45.511495
# 10952              211548_s_at"  39.885743
# 3439               203913_s_at"  39.213689
# 19338              219975_x_at"  38.786768
# 9477                 209992_at"  36.792002
# 5370                 205844_at"  36.757702
# 5702               206177_s_at"  35.538623
# 19032                219669_at"  35.401498
# 3440               203914_x_at"  35.133532
# 8786               209293_x_at"  34.937390
# 22240  AFFX-HUMRGE/M10098_M_at"  34.897467
# 16040              216671_x_at"  31.861675
# 9445                 209960_at"  28.876673
# 16806                217442_at"  28.460255
# 2680                 203153_at"  27.864647
# 5975                 206450_at"  27.230384